In [ ]:
%load_ext autoreload
%autoreload 2

#Chargement des données


In [ ]:
!pip install datasets bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset, Dataset
from typing import Dict, Union
from functools import partial
from itertools import chain

In [ ]:
def get_dataset(
    dataset_path="lavita/medical-qa-datasets",
    dataset_name="chatdoctor_healthcaremagic"
):
    """
    Process a SQL dataset by filtering and preparing context and instructions.

    Args:
        dataset_name (str): Name of the dataset to load from Hugging Face Hub
        source_filter (str): Prefix to filter dataset sources

    Returns:
        processed_dataset: Dataset with added context and refined instructions
    """
    # Load dataset from the hub and filter dataset based on source
    dataset = load_dataset(path=dataset_path, name=dataset_name, split="train")

    dataset_size = len(dataset)
    print(f"Dataset size: {dataset_size}")
    return dataset

In [ ]:
dataset = get_dataset()

In [ ]:
dataset = dataset.select(range(1000))

# Préparation des données

In [ ]:
from transformers import AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
model_id = "Qwen/Qwen2-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
def split_dataset_by_size(
    dataset: Dataset,
    train_size: Union[int, float] = 0.7,
    eval_size: Union[int, float] = 0.2,
    test_size: Union[int, float] = 0.1,
    train_seed: int = 42,
    eval_seed: int = 42
) -> Dict[str, Dataset]:
    """
    Split dataset deterministically into train, eval, and test sets with flexible sizing.

    Args:
        dataset (Dataset): Processed dataset to split
        train_size (int or float): Number or proportion of samples for training
        eval_size (int or float): Number or proportion of samples for evaluation
        test_size (int or float): Number or proportion of samples for testing
        train_seed (int): Seed for selecting the fixed train dataset
        eval_seed (int): Seed for selecting eval and test datasets

    Returns:
        Dict containing train, eval, and test datasets
    """
    # Total dataset size
    total_size = len(dataset)

    # Convert proportional sizes to actual numbers if needed
    def convert_to_size(size_param, total):
        if isinstance(size_param, float):
            return int(total * size_param)
        return size_param

    train_size = convert_to_size(train_size, total_size)
    eval_size = convert_to_size(eval_size, total_size)
    test_size = convert_to_size(test_size, total_size)

    # Validate sizes
    if train_size + eval_size + test_size > total_size:
        raise ValueError(f"Combined sizes ({train_size + eval_size + test_size}) exceed total dataset size ({total_size})")

    # Shuffle the entire dataset for consistent splitting
    shuffled_dataset = dataset.shuffle(seed=train_seed)

    # Select train dataset
    train_dataset = shuffled_dataset.select(range(train_size))

    # Select remaining dataset (excluding train)
    remaining_dataset = shuffled_dataset.select(range(train_size, total_size))

    # Shuffle remaining dataset with eval seed
    remaining_dataset = remaining_dataset.shuffle(seed=eval_seed)

    # Select eval and test datasets from remaining
    eval_dataset = remaining_dataset.select(range(eval_size))
    test_dataset = remaining_dataset.select(range(eval_size, eval_size + test_size))

    # Print split information
    print(f"Dataset Split Information:")
    print(f"Total samples: {total_size}")
    print(f"Train samples: {len(train_dataset)} ({train_size})")
    print(f"Eval samples: {len(eval_dataset)} ({eval_size})")
    print(f"Test samples: {len(test_dataset)} ({test_size})")

    return {
        "dataset": shuffled_dataset,
        "train": train_dataset,
        "eval": eval_dataset,
        "test": test_dataset
    }

In [ ]:
dataset_splits = split_dataset_by_size(dataset=dataset)

In [ ]:
# Extract train, eval, and test datasets
train_dataset = dataset_splits["train"]
eval_dataset = dataset_splits["eval"]
test_dataset = dataset_splits["test"]

In [ ]:
def chunk(sample, chunk_length=2048):
    """
    Chunk tokenized dataset into fixed-length sequences

    Args:
        sample (dict): Tokenized dataset sample
        chunk_length (int): Length of chunks to create

    Returns:
        dict: Chunked dataset sample
    """
    # Define global remainder variable to save remainder from batches to use in next batch
    global remainder

    # Concatenate all texts and add remainder from previous batch
    concatenated_examples = {k: list(chain(*sample[k])) for k in sample.keys()}
    concatenated_examples = {k: remainder[k] + concatenated_examples[k] for k in concatenated_examples.keys()}

    # Get total number of tokens for batch
    batch_total_length = len(concatenated_examples[list(sample.keys())[0]])

    # Get max number of chunks for batch
    if batch_total_length >= chunk_length:
        batch_chunk_length = (batch_total_length // chunk_length) * chunk_length

    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_length] for i in range(0, batch_chunk_length, chunk_length)]
        for k, t in concatenated_examples.items()
    }

    # Add remainder to global variable for next batch
    remainder = {k: concatenated_examples[k][batch_chunk_length:] for k in concatenated_examples.keys()}

    # Prepare labels
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
def format_dolly(sample):
    """
    Format sample in Dolly-style prompt template

    Args:
        sample (dict): Dataset sample with instruction, input, and output

    Returns:
        str: Formatted prompt
    """
    instruction = f"### Instruction\n{sample['instruction']}"
    input = f"### Input\n{sample['input']}" if len(sample["input"]) > 0 else None
    response = f"### Answer\n{sample['output']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [instruction, input, response] if i is not None])
    return prompt

In [ ]:
def template_dataset(sample):
    """
    Add formatted text to dataset sample

    Args:
        sample (dict): Dataset sample

    Returns:
        dict: Sample with added text field
    """
    sample["text"] = f"{format_dolly(sample)}{tokenizer.eos_token}"
    return sample

In [ ]:
def process_dataset(dataset: Dataset, chunk_length: int = 2048) -> Dataset:
  # Initialize remainder dictionary for chunking
  global remainder
  remainder = {"input_ids": [], "attention_mask": [], "token_type_ids": []}

  # Prepare train dataset
  # Apply prompt template per sample
  dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))

  # Tokenize and chunk train dataset
  lm_dataset = dataset.map(
      lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
  ).map(
      partial(chunk, chunk_length=chunk_length),
      batched=True,
  )

  # Print total number of samples
  print(f"Total train samples: {len(lm_dataset)}")

  return lm_dataset

In [ ]:
lm_dataset_train = process_dataset(train_dataset)
lm_dataset_eval = process_dataset(eval_dataset)

# PEFT

In [ ]:
import sqlite3
from datasets import load_from_disk
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training
from peft.tuners.lora import LoraLayer
import bitsandbytes as bnb
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, default_data_collator, Trainer, TrainingArguments, AutoTokenizer
import torch

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    use_cache=False,
    device_map="auto",
    quantization_config=bnb_config
)

In [ ]:
def find_all_linear_names(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, bnb.nn.Linear4bit):
            names = name.split(".")
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if "lm_head" in lora_module_names:  # needed for 16-bit
        lora_module_names.remove("lm_head")
    return list(lora_module_names)

In [ ]:
modules = find_all_linear_names(model)

In [ ]:
peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=modules,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

In [ ]:
model.enable_input_require_grads()

In [ ]:
def create_peft_model(model, peft_config, gradient_checkpointing=True, bf16=True):

    # prepare int-4 model for training
    model = prepare_model_for_kbit_training(
        model, use_gradient_checkpointing=gradient_checkpointing
    )
    if gradient_checkpointing:
        model.gradient_checkpointing_enable()

    # get lora target modules
    modules = find_all_linear_names(model)
    print(f"Found {len(modules)} modules to quantize: {modules}")

    model = get_peft_model(model, peft_config)

    # pre-process the model by upcasting the layer norms in float 32 for
    for name, module in model.named_modules():
        if isinstance(module, LoraLayer):
            if bf16:
                module = module.to(torch.bfloat16)
        if "norm" in name:
            module = module.to(torch.float32)
        if "lm_head" in name or "embed_tokens" in name:
            if hasattr(module, "weight"):
                if bf16 and module.weight.dtype == torch.float32:
                    module = module.to(torch.bfloat16)

    model.print_trainable_parameters()
    return model

In [ ]:
model = create_peft_model(
    model=model,
    peft_config=peft_config,
    gradient_checkpointing=True,
    bf16=True if torch.cuda.get_device_capability()[0] == 8 else False
)

In [ ]:
model.print_trainable_parameters()

In [ ]:
training_args = TrainingArguments(
    output_dir="./tmp/qwen2",
    per_device_train_batch_size=3,
    bf16=True if torch.cuda.get_device_capability()[0] == 8 else False,
    learning_rate=2e-4,
    num_train_epochs=3,
    gradient_checkpointing=True,
    logging_dir="./tmp/qwen2/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
    seed = 3578,
)

In [ ]:
# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset_train,
    eval_dataset=lm_dataset_eval,
    processing_class=tokenizer,
    data_collator=default_data_collator
)

In [ ]:
# Start training
trainer.train()
model.save_pretrained("./finetuned_llm/qwen2")

In [ ]:
torch.cuda.empty_cache()

# Inférence

In [ ]:
import pandas as pd
import sqlite3
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_id = "Qwen/Qwen2-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
import bitsandbytes as bnb
from transformers import BitsAndBytesConfig
import torch

In [ ]:
#model = AutoModelForCausalLM.from_pretrained(
#    "./finetuned_llm/qwen2",
#    torch_dtype="auto",
#    device_map="auto",
#    max_memory={0: "5GB"}
#)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    use_cache=False,
    device_map="auto",
    quantization_config=bnb_config
)

In [ ]:
from abc import abstractmethod, ABC
import json

from transformers import AutoTokenizer
from peft import AutoPeftModelForCausalLM

## Abstract class acting as port for language model
class LargeLanguageModel(ABC):

    @abstractmethod
    def generate(query: str) -> str:
        pass

## Qwen2 language model class based on languageModel port
class LLM(LargeLanguageModel):
    def __init__(self, model: AutoPeftModelForCausalLM, tokenizer: AutoTokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def generate(self, natural_query: str) -> str:

        text = self.tokenizer.apply_chat_template(
            natural_query,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)

        generated_ids = self.model.generate(
            **model_inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.9,
            top_k=50,
            top_p=0.9
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        return self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0].split("SQLQuery:")[-1]

## prepare query with prompt
def prepare_query(natural_query: str, template: str) -> str:
    # template = json.load(open(path_to_prompt_template, "r"))
    #prompt = f"{template['role']} \n" # --{template['context']}"
    messages = [
        {"role": "system", "content": template},
        {"role": "user", "content": natural_query}
    ]
    return messages


## generate answer from natural query with given llm
def generate_answer(natural_query: str, language_model: LargeLanguageModel) -> str:
    return language_model.generate(natural_query)

In [ ]:
llm = LLM(model=model, tokenizer=tokenizer)

In [ ]:
role = " ".join(test_dataset[0:1]["instruction"])
question = " ".join(test_dataset[0:1]["input"])

In [ ]:
template = json.dumps(
  {
    "role": role,
  }
)

In [ ]:
query = prepare_query(question, template=role)
answer_generated = generate_answer(query, llm)

# Évaluation

In [ ]:
!pip install evaluate

In [ ]:
## BLEU score

In [ ]:
import evaluate

In [ ]:
answer = " ".join(test_dataset[0:1]["output"])

## BLEU score

In [ ]:
bleu = evaluate.load("bleu")

results = bleu.compute(predictions=[answer_generated], references=[answer])
print(results)
results = bleu.compute(predictions=[answer_generated], references=[answer], tokenizer=tokenizer)
print(results)

**In summary for BLEU**: The very low BLEU score, combined with the 0 precision values for n-grams greater than 1 and the high length ratio, indicates that the generated text is significantly different from the expected answer, and the overlap is primarily at the individual word level, not in terms of meaningful phrases or sentences.

## ROUGE score

In [ ]:


import evaluate


rouge = evaluate.load("rouge") # Use ROUGE for text generation tasks

# Compute ROUGE score (more suitable for text generation)
rouge_result = rouge.compute(predictions=[answer_generated], references=[answer])
print("ROUGE:", rouge_result)





ROUGE: {'rouge1': 0.1595092024539877, 'rouge2': 0.012422360248447208, 'rougeL': 0.12269938650306748, 'rougeLsum': 0.12269938650306748}


**In summary for ROUGE:** The ROUGE scores are also low, confirming that there's limited similarity between the generated and expected answers. The low rouge2 score is particularly notable, as it highlights a lack of shared phrases, further suggesting that the generated text might not be expressing the desired information effectively.

## Human Evaluation

In [ ]:
# Human Evaluation Metrics (Qualitative)

# 1.  Fluency: Assess the grammatical correctness and readability of the generated answer.
#     *   Score on a scale (e.g., 1-5, where 5 is perfect fluency).
#     *   Example: "The generated answer is grammatically correct and easy to understand."

# 2.  Relevance: Evaluate how well the generated answer addresses the question or prompt.
#     *   Score on a scale (e.g., 1-5, where 5 is perfectly relevant).
#     *   Example: "The generated answer directly and completely addresses all aspects of the question."

# 3.  Accuracy: Determine the factual correctness of the generated answer.
#     *   Score on a scale (e.g., 1-5, where 5 is perfectly accurate).
#     *   Example: "The information provided in the answer is factually correct and verifiable."

# 4.  Comprehensiveness: Assess whether the generated answer covers all relevant aspects of the question.
#     *   Score on a scale (e.g., 1-5, where 5 indicates complete coverage).
#     *   Example:  "The response provides all the necessary details in a concise manner."



In [ ]:

# Generate evaluation data
evaluation_data = []
for i in range(len(test_dataset)):
  role = " ".join(test_dataset[i:i+1]["instruction"])
  question = " ".join(test_dataset[i:i+1]["input"])
  answer = " ".join(test_dataset[i:i+1]["output"])

  template = json.dumps({"role": role})
  query = prepare_query(question, template=role)
  answer_generated = generate_answer(query, llm)
  evaluation_data.append({
      "question": question,
      "answer": answer,
      "generated_answer": answer_generated
  })

# Evaluate the model (example using BLEU)
bleu = evaluate.load("bleu")

for item in evaluation_data:
    results = bleu.compute(predictions=[item["generated_answer"]], references=[item["answer"]])
    print(f"Question: {item['question']}")
    print(f"Generated Answer: {item['generated_answer']}")
    print(f"Expected Answer: {item['answer']}")
    print(f"BLEU Score: {results}")
    print("-" * 20)


Question: I have been trying to figure out why my legs are hurting so much, about an 8-10. Also my knees are hurting and are swollen all the time lately and are week, they give out a lot. I m recoding from thoriac back surgery, my surgeon cut a couple of chunks out and straightened my spine then fused from T8 to L1 were he tied into the 14 screws he placed while fussing L1-my sacrum in 2009, I now have 26 total screws. I m wondering if I injured my femoral nerve because my groin and inside of my legs hurt all the down to my ankles, along with my knee and leg pain?
Generated Answer: Hi! Thanks for choosing Chat Doctor for your health related queries. As per your clinical history there is possibility of pinched nerve root causing that type of pain. But due to the presence of large number of screws in spine as well as the involvement of sacroiliac joint I can't rule out disc prolapse, intervertebral disk degenerative disease and other conditions like Ankylosing spondylitis, Fibromyalgia w

## LLM as a judge


In [ ]:

import json

class LLMBasedJudge:
    def __init__(self, llm: LargeLanguageModel, prompt_template: str):
        self.llm = llm
        self.prompt_template = prompt_template

    def evaluate(self, generated_answer: str, reference_answer: str) -> Dict[str, float]:
        prompt = self.prompt_template.format(
            generated_answer=generated_answer, reference_answer=reference_answer
        )
        # Modify prompt to a list of dictionaries with 'role' and 'content'
        prompt = [{"role": "user", "content": prompt}]  # Assuming user role for the prompt
        evaluation = self.llm.generate(prompt)  # Pass the modified prompt
        # Assuming the LLM returns a JSON with scores
        try:
            print(f"Raw evaluation output: {evaluation}")  # Debugging line
            scores = json.loads(evaluation)
            return scores
        except json.JSONDecodeError:
            print(f"Warning: Invalid JSON returned by LLM: {evaluation}")
            return {"overall_score": 0.0}  # or handle the error differently

In [ ]:
# Example usage (assuming you have a suitable prompt_template)
prompt_template = """
    ## Medical Answer Evaluation

    **Instructions:**

    You are a medical specialist. Carefully compare the "Generated Answer" to the "Reference Answer" and provide scores for the following criteria:

    * **Fluency (1-5):** How clear, concise, and grammatically correct is the generated answer?
        * 5: Flawless, natural language.
        * 1: Incoherent or contains major grammatical errors.
    * **Relevance (1-5):** How well does the generated answer address the implied medical question?
        * 5: Completely addresses the core medical concerns.
        * 1: Utterly irrelevant or off-topic.
    * **Accuracy (1-5):**  How factually correct is the generated answer based on established medical knowledge?
        * 5: Completely accurate and evidence-based.
        * 1: Contains significant factual errors or misinformation.
    * **Comprehensiveness (1-5):** Does the generated answer provide sufficient detail and context for a medical inquiry?
        * 5: Thoroughly covers all relevant aspects.
        * 1: Superficial or lacking crucial information.
    * **Neutrality

    **Generated Answer:** {generated_answer}
    **Reference Answer:** {reference_answer}

    **Output:**

    Return your evaluation as a JSON object with the following format:
    {{ "fluency": 4, "relevance": 5, "accuracy": 3, "comprehensiveness": 4, "overall_score": 4 }}
  **Note:** Please ensure your evaluation is unbiased and grounded in your medical expertise. Provide an overall score (1-5) reflecting your holistic judgment of the generated answer's quality and usefulness in a medical context.
"""



judge = LLMBasedJudge(llm=llm, prompt_template=prompt_template)

evaluation_results = []
for item in evaluation_data:
    scores = judge.evaluate(item["generated_answer"], item["answer"])
    evaluation_results.append({
        "question": item['question'],
        "generated_answer": item["generated_answer"],
        "reference_answer": item["answer"],
        "scores": scores
    })


Raw evaluation output: { "fluency": 4, "relevance": 5, "accuracy": 3, "comprehensiveness": 4, "overall_score": 4 }
Raw evaluation output: { "fluency": 4, "relevance": 5, "accuracy": 3, "comprehensiveness": 4, "overall_score": 4 }
Raw evaluation output: { "fluency": 4, "relevance": 5, "accuracy": 3, "comprehensiveness": 4, "overall_score": 4 }
Raw evaluation output: { "fluency": 4, "relevance": 5, "accuracy": 3, "comprehensiveness": 4, "overall_score": 4 }
Raw evaluation output: ```json
{
  "fluency": 4,
  "relevance": 5,
  "accuracy": 3,
  "comprehensiveness": 4,
  "overall_score": 4
}
```
{
  "fluency": 4,
  "relevance": 5,
  "accuracy": 3,
  "comprehensiveness": 4,
  "overall_score": 4
}
```
Raw evaluation output: {
  "fluency": 4,
  "relevance": 5,
  "accuracy": 3,
  "comprehensiveness": 4,
  "overall_score": 4
}
Raw evaluation output: {
  "fluency": 4,
  "relevance": 5,
  "accuracy": 3,
  "comprehensiveness": 4,
  "overall_score": 4
}
Raw evaluation output: ```json
{
  "fluency": 4

In [ ]:
# Print or save the evaluation results
from pprint import pprint
for result in evaluation_results:
  pprint(result)

{'generated_answer': 'Hi! Thanks for choosing Chat Doctor for your health '
                     'related queries. As per your clinical history there is '
                     'possibility of pinched nerve root causing that type of '
                     'pain. But due to the presence of large number of screws '
                     'in spine as well as the involvement of sacroiliac joint '
                     "I can't rule out disc prolapse, intervertebral disk "
                     'degenerative disease and other conditions like '
                     'Ankylosing spondylitis, Fibromyalgia which can cause '
                     'such kind of pain as well. In my opinion, it would be '
                     'beneficial to get clinically examined by an orthopedic '
                     'surgeon who can thoroughly evaluate your lower limb and '
                     'spine problems along with performing some investigations '
                     'required. Hope this helps. Take care. Plea